# Classification for Machine Learning




In [1]:
import sys
sys.path.insert(1, '../../crystal-code-tools/crystal_functions/crystal_functions/')

from file_readwrite import Crystal_output, write_cry_properties, Crystal_input, write_cry_gui,write_cry_input, Density
from execute import runcry
from convert import cry_gui2pmg

from pymatgen.io.ase import AseAtomsAdaptor
from pymatgen.io.cif import CifWriter
from ase.visualize import view

from pymatgen.symmetry.analyzer import SpacegroupAnalyzer
from pymatgen.core.surface import SlabGenerator
import numpy as np
import pandas as pd

<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [2]:
new_atom = 'Ca'

### Read the structures from the output files and replace with new_atom
This cell generates the structures list. This contains all the 4023 structures as pymatgen structure objects

In [8]:
cry_output = Crystal_output('data/classification/ml/mgo_confrand.out')
cry_output.config_analysis()


original_structure = cry_gui2pmg('data/classification/ml/mgo_confrand.gui')
structures = []
mg_atoms = []
ca_atoms = []
for j,substitutions in enumerate(cry_output.atom_type2):
    new_structure = original_structure.copy()
    for i in substitutions:
        new_structure.replace(i-1,new_atom)
    structures.append(new_structure)
    mg_atoms.append((np.array(cry_output.atom_type1[j])-1).tolist())
    ca_atoms.append((np.array(cry_output.atom_type2[j])-1).tolist())

### Shells

In [26]:
shells = np.unique(np.round(structures[0].distance_matrix[0],decimals=6),return_counts=True)[0].tolist()
display(shells)
np.unique(np.round(structures[0].distance_matrix[0],decimals=6),return_counts=True)[1]

[0.0,
 2.105,
 2.97692,
 3.645967,
 4.21,
 4.706923,
 5.156176,
 5.953839,
 6.315,
 6.656594,
 6.981495,
 7.291934,
 7.589685,
 7.876189,
 8.679137,
 8.930759,
 9.175482,
 9.873325,
 10.937901]

array([ 1,  6, 12,  8,  6, 24, 24, 12, 27, 12, 12,  8, 12, 24, 12,  3,  6,
        6,  1])

In [22]:
view(AseAtomsAdaptor().get_atoms(structures[0]))

<Popen: returncode: None args: ['/Users/brunocamino/miniconda3/envs/test_env...>

<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


### Site surrounding

#### Second shell

In [7]:
unique_atom_type = []

# The magic happens here

# Create a list of the unique densities

In [29]:

#structure = cry_gui2pmg('data/classification/LTS_2_B3LYP_Ahl_b.gui')


#structure = cry_gui2pmg('data/classification/mgo_scel.gui')
structure = structures[0]

#How many first/second neighbours can an atom have?
max_n_first_shell = 6
max_n_second_shell = 12
max_n_third_shell = 8

#This is the final one (list of lists)

#list of lists. Each element has 0 central atom specie, 1: surrounding atom species (-1 if there are not enough neighbours
#for example: [8, 12, 12, 12, -1]. Unique elements only (unique to all structures)
unique_neighbours = []

#list of lists. Each element has 0 central atom specie, 1: surrounding atom species (-1 if there are not enough neighbours
#for example: [8, 12, 12, 12, -1]. All elements
first_neigh_atoms_all = []
#list of lists. Each element has 0 central atom index, 1: surrounding atom index (-1 if there are not enough neighbours
#All elements
first_neigh_all = []

#list of lists. Each element has 0 central atom specie, 1: surrounding atom species (-1 if there are not enough neighbours
#for example: [8, 12, 12, 12, -1]. Unique elements only (unique to this structure)
first_neigh_atoms_unique = []
#list of lists. Each element has 0 central atom index, 1: surrounding atom index (-1 if there are not enough neighbours
# Unique elements only (unique to this structure)
first_neigh_unique = []

#structure = mgo
#structure = structures[126]


for i,atom in enumerate(structure.sites):
    first_neigh = []
    first_neigh_atoms = []
    
    second_neigh = []
    second_neigh_atoms = []
    
    third_neigh = []
    third_neigh_atoms = []
    for j,atom in enumerate(structure.sites):        

        
        #Check is the atom j is in the first shell in the unit cell
        if abs(structure.sites[i].distance(structure.sites[j]) - shells[1]) < 0.1:
            first_neigh.append(j)
            first_neigh_atoms.append(structure.atomic_numbers[j])
        if abs(structure.sites[i].distance(structure.sites[j]) - shells[2]) < 0.1:
            second_neigh.append(j)
            second_neigh_atoms.append(structure.atomic_numbers[j])
    
        if abs(structure.sites[i].distance(structure.sites[j]) - shells[3]) < 0.1:
            third_neigh.append(j)
            third_neigh_atoms.append(structure.atomic_numbers[j])
            
    #print(i,j,first_neigh_atoms)
    sort_list = np.argsort(np.array(first_neigh_atoms))
    sort_list_second = np.argsort(np.array(second_neigh_atoms))
    sort_list_third = np.argsort(np.array(third_neigh_atoms))
    
    #First neighbour index for the i-th atom sorted
    first_neigh = (np.array(first_neigh)[sort_list]).tolist()
    #First neighbour species for the i-th atom sorted
    first_neigh_atoms = (np.array(first_neigh_atoms)[sort_list]).tolist()
    
    #Second neighbour index for the i-th atom sorted
    second_neigh = (np.array(second_neigh)[sort_list_second]).tolist()
    #Second neighbour species for the i-th atom sorted
    second_neigh_atoms = (np.array(second_neigh_atoms)[sort_list_second]).tolist()
    
    #T neighbour index for the i-th atom sorted
    third_neigh = (np.array(third_neigh)[sort_list_third]).tolist()
    #T neighbour species for the i-th atom sorted
    third_neigh_atoms = (np.array(third_neigh_atoms)[sort_list_third]).tolist()

    #Prepend the atom index to the first neighbours list
    first_neigh.insert(0,i)
    #Prepend the atom specie to the first neighbours list
    first_neigh_atoms.insert(0,structure.atomic_numbers[i])
    
    #Add -1 to first_neigh and first_neigh_atoms to reach the same lenght for all vectors
    if len(first_neigh) < max_n_first_shell+1:
        n = max_n_first_shell+1 - len(first_neigh)
        first_neigh.extend([-1]*n)
        first_neigh_atoms.extend([-1]*n)
    
    #Same for the second shell
    if len(second_neigh) < max_n_second_shell:
        n = max_n_second_shell+1 - len(second_neigh)
        second_neigh.extend([-1]*n)
        second_neigh_atoms.extend([-1]*n)
        
    #Same for the third shell
    if len(third_neigh) < max_n_third_shell:
        n = max_n_third_shell+1 - len(third_neigh)
        third_neigh.extend([-1]*n)
        third_neigh_atoms.extend([-1]*n)
        
    #TMP: append second to first neigh
    first_neigh.extend(second_neigh)
    first_neigh_atoms.extend(second_neigh_atoms)
    first_neigh.extend(third_neigh)
    first_neigh_atoms.extend(third_neigh_atoms)
    
    
    
    first_neigh_atoms_all.append(first_neigh_atoms) 
    first_neigh_all.append(first_neigh)
#print(first_neigh_atoms_all[0])
first_neigh_atoms_unique.extend(np.unique(first_neigh_atoms_all,axis=0,return_index=True)[0].tolist())
index = np.unique(first_neigh_atoms_all,axis=0,return_index=True)[1]
first_neigh_unique.extend((np.array(first_neigh_all)[index]).tolist())
              

[0.0, 2.105, 2.97692, 3.645967, 4.21, 4.706923, 5.156176, 5.953839, 6.315, 6.656594, 6.981495, 7.291934, 7.589685, 7.876189, 8.679137, 8.930759, 9.175482, 9.873325, 10.937901]
[20, 8, 8, 8, 8, 8, 8, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 8, 8, 8, 8, 8, 8, 8, 8]


In the 2D array first_neigh_atoms_all each row corresponds to an atom and the array contains:
- 0 = atom number of the atom whose neighbours I'm analysing
- 1:6 = atomic numbers of the atoms in the first coord shell
- 7:19 = atomic numbers of the atoms in the second coord shell
- 20:28 = atomic numbers of the atoms in the third coord shell

In [46]:
for i in first_neigh_atoms_all:
    display(i)

[20,
 8,
 8,
 8,
 8,
 8,
 8,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8]

[20,
 8,
 8,
 8,
 8,
 8,
 8,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8]

[20,
 8,
 8,
 8,
 8,
 8,
 8,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8]

[20,
 8,
 8,
 8,
 8,
 8,
 8,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8]

[20,
 8,
 8,
 8,
 8,
 8,
 8,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8]

...

[8,
 12,
 12,
 12,
 12,
 12,
 12,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12]

[8,
 12,
 12,
 12,
 12,
 12,
 12,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12]

[8,
 12,
 12,
 12,
 12,
 12,
 12,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12]

[8,
 12,
 12,
 12,
 12,
 12,
 12,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12]

[8,
 12,
 12,
 12,
 12,
 12,
 12,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12]